In [5]:
import csv
import numpy as np
import pandas as pd
import random

from scipy.stats import randint as sp_randint
#from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV




In [3]:
train = pd.read_csv('data.csv', sep=",")
test_data = pd.read_csv('quiz.csv', sep=",")

train_data = train.iloc[:,:-1]
train_labels = train.iloc[:,-1]

train_data, holdout_data, train_labels, holdout_label = train_test_split(train_data, train_labels, 
                                                                         test_size=0.2, random_state=1)
all_data = pd.concat([train_data, test_data])
train_obs = len(train_data)
test_obs = len(test_data)

# Change dtype of categorical columns
categorical_columns = ['0','5','7','8','9','14','16','17','18','20','23','25','26','56','57','58']
for i in range(0,len(categorical_columns)):
    all_data[categorical_columns[i]] = all_data[categorical_columns[i]].astype('category')
    
# Only numerical data
print('Processing numerical data...')
all_data_num = all_data.drop(categorical_columns, axis=1)
train_data_num = all_data_num.iloc[0:train_obs,]
test_data_num = all_data_num.iloc[train_obs:,]

## Feature engineering
train_data_num['div'] = (train_data_num.loc[:,'60'] / train_data_num.loc[:,'59'])
train_data_num['div'] = train_data_num['div'].fillna(0)
test_data_num['div'] = (test_data_num.loc[:,'60'] / test_data_num.loc[:,'59'])
test_data_num['div'] = test_data_num['div'].fillna(0)

# Only categorical data
print('Processing categorical data...')
all_data_cat = pd.get_dummies(all_data[categorical_columns])
train_data_cat = all_data_cat.iloc[0:train_obs,]
test_data_cat = all_data_cat.iloc[train_obs:,]

# Combine the two
print('Combining data...')
train_data_combo = pd.concat([train_data_num, train_data_cat], axis=1)
test_data_combo = pd.concat([test_data_num, test_data_cat], axis=1)

print('Processing finished!')

Processing numerical data...


/Users/PS/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/PS/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/PS/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

Processing categorical data...
Combining data...
Processing finished!


In [6]:
params = {
    'n_estimators': sp_randint(35, 100),
    'max_features': sp_randint(750, 3000),
    'min_samples_split': sp_randint(2, 5),
    'min_samples_leaf': sp_randint(1, 5),
    'random_state': [1],
    'n_jobs': [-1]
}
classifier = ExtraTreesClassifier()
search_results2 = RandomizedSearchCV(estimator = classifier,
                                        param_distributions = params,
                                        n_iter = 30, n_jobs = 1,
                                        cv = 3, verbose = 3 )
# GridLogiClassifier = GridSearchCV(estimator = classifier, param_grid=params, n_jobs=1, cv=4, verbose=2)
search_results2.fit(train_data_combo, train_labels)

Fitting 3 folds for each of 30 candidates, totalling 90 fits
[CV] n_jobs=-1, min_samples_leaf=1, n_estimators=75, min_samples_split=2, random_state=1, max_features=1410 
[CV]  n_jobs=-1, min_samples_leaf=1, n_estimators=75, min_samples_split=2, random_state=1, max_features=1410, score=0.941136 - 7.6min
[CV] n_jobs=-1, min_samples_leaf=1, n_estimators=75, min_samples_split=2, random_state=1, max_features=1410 
[CV]  n_jobs=-1, min_samples_leaf=1, n_estimators=75, min_samples_split=2, random_state=1, max_features=1410, score=0.940869 - 7.1min
[CV] n_jobs=-1, min_samples_leaf=1, n_estimators=75, min_samples_split=2, random_state=1, max_features=1410 
[CV]  n_jobs=-1, min_samples_leaf=1, n_estimators=75, min_samples_split=2, random_state=1, max_features=1410, score=0.942789 - 7.4min
[CV] n_jobs=-1, min_samples_leaf=2, n_estimators=43, min_samples_split=2, random_state=1, max_features=2500 
[CV]  n_jobs=-1, min_samples_leaf=2, n_estimators=43, min_samples_split=2, random_state=1, max_featur

[Parallel(n_jobs=1)]: Done  31 tasks       | elapsed: 210.6min


[CV]  n_jobs=-1, min_samples_leaf=2, n_estimators=56, min_samples_split=2, random_state=1, max_features=1168, score=0.940721 - 4.3min
[CV] n_jobs=-1, min_samples_leaf=2, n_estimators=56, min_samples_split=2, random_state=1, max_features=1168 
[CV]  n_jobs=-1, min_samples_leaf=2, n_estimators=56, min_samples_split=2, random_state=1, max_features=1168, score=0.942877 - 4.3min
[CV] n_jobs=-1, min_samples_leaf=4, n_estimators=56, min_samples_split=2, random_state=1, max_features=817 
[CV]  n_jobs=-1, min_samples_leaf=4, n_estimators=56, min_samples_split=2, random_state=1, max_features=817, score=0.936583 - 3.4min
[CV] n_jobs=-1, min_samples_leaf=4, n_estimators=56, min_samples_split=2, random_state=1, max_features=817 
[CV]  n_jobs=-1, min_samples_leaf=4, n_estimators=56, min_samples_split=2, random_state=1, max_features=817, score=0.936434 - 3.4min
[CV] n_jobs=-1, min_samples_leaf=4, n_estimators=56, min_samples_split=2, random_state=1, max_features=817 
[CV]  n_jobs=-1, min_samples_leaf

[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed: 715.4min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params={}, iid=True, n_iter=30, n_jobs=1,
          param_distributions={'n_jobs': [-1], 'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x10d0a6650>, 'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x10c2b6bd0>, 'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x10c4d3050>, 'random_state': [1], 'max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1074fff90>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring=None, verbose=3)

# With first engineered feature

In [7]:
search_results2.best_score_

0.94197242507563883

In [8]:
search_results2.best_params_

{'max_features': 1611,
 'min_samples_leaf': 2,
 'min_samples_split': 3,
 'n_estimators': 99,
 'n_jobs': -1,
 'random_state': 1}

In [9]:
search_results2.grid_scores_

[mean: 0.94160, std: 0.00085, params: {'n_jobs': -1, 'min_samples_leaf': 1, 'n_estimators': 75, 'min_samples_split': 2, 'random_state': 1, 'max_features': 1410},
 mean: 0.94126, std: 0.00125, params: {'n_jobs': -1, 'min_samples_leaf': 2, 'n_estimators': 43, 'min_samples_split': 2, 'random_state': 1, 'max_features': 2500},
 mean: 0.94138, std: 0.00071, params: {'n_jobs': -1, 'min_samples_leaf': 1, 'n_estimators': 76, 'min_samples_split': 4, 'random_state': 1, 'max_features': 1913},
 mean: 0.93885, std: 0.00134, params: {'n_jobs': -1, 'min_samples_leaf': 4, 'n_estimators': 95, 'min_samples_split': 2, 'random_state': 1, 'max_features': 2407},
 mean: 0.94002, std: 0.00084, params: {'n_jobs': -1, 'min_samples_leaf': 3, 'n_estimators': 39, 'min_samples_split': 2, 'random_state': 1, 'max_features': 2717},
 mean: 0.93749, std: 0.00087, params: {'n_jobs': -1, 'min_samples_leaf': 4, 'n_estimators': 57, 'min_samples_split': 2, 'random_state': 1, 'max_features': 984},
 mean: 0.94015, std: 0.00111,

## First Run (coarse)

In [5]:
search_results.best_score_

0.94095733672353132

In [6]:
search_results.grid_scores_

[mean: 0.92432, std: 0.00103, params: {'n_jobs': -1, 'min_samples_leaf': 4, 'n_estimators': 68, 'min_samples_split': 2, 'random_state': 1, 'max_features': 2629, 'max_depth': 20},
 mean: 0.92386, std: 0.00149, params: {'n_jobs': -1, 'min_samples_leaf': 4, 'n_estimators': 45, 'min_samples_split': 2, 'random_state': 1, 'max_features': 2456, 'max_depth': 20},
 mean: 0.93975, std: 0.00134, params: {'n_jobs': -1, 'min_samples_leaf': 3, 'n_estimators': 95, 'min_samples_split': 4, 'random_state': 1, 'max_features': 1770, 'max_depth': None},
 mean: 0.93868, std: 0.00133, params: {'n_jobs': -1, 'min_samples_leaf': 3, 'n_estimators': 23, 'min_samples_split': 3, 'random_state': 1, 'max_features': 1371, 'max_depth': None},
 mean: 0.88300, std: 0.00219, params: {'n_jobs': -1, 'min_samples_leaf': 2, 'n_estimators': 26, 'min_samples_split': 3, 'random_state': 1, 'max_features': 1906, 'max_depth': 10},
 mean: 0.94083, std: 0.00082, params: {'n_jobs': -1, 'min_samples_leaf': 1, 'n_estimators': 40, 'min_